In [8]:
import numpy as np
import pandas as pd
import os
import joblib

In [12]:
os.getcwd()

'/Users/nataliamarko/Documents/Kaggle/home_credit_risk_score'

In [16]:
# Load the data
test_data = joblib.load('/Users/nataliamarko/Documents/Kaggle/home_credit_risk_score/temp_data/test_data_pp.pkl')
train_data = joblib.load('/Users/nataliamarko/Documents/Kaggle/home_credit_risk_score/temp_data/train_data_pp.pkl')

In [18]:
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from lightgbm import LGBMClassifier
import gc


# Train the model using LightGBM
y = train_data['target']
X = train_data.drop(['target', 'case_id', 'WEEK_NUM'], axis=1)
weeks = train_data["WEEK_NUM"]

# Standardize the data before splitting
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into train and test sets
X_train_, X_test, y_train_, y_test = train_test_split(X, y, 
                                                      test_size=0.2, random_state=42)

# Further split train data into train and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train_, y_train_, 
                                                      test_size=0.2, random_state=42)

cv = GroupKFold(n_splits=2)

lgbm_model = LGBMClassifier(force_col_wise=True, 
                            random_state=42)

valid_roc_aucs = []
for idx_train, idx_valid in cv.split(X_scaled, y, groups=weeks):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
    # Train the model
    lgbm_model.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_metric='auc',
        callbacks=[lgb.early_stopping(stopping_rounds=20, verbose=0)]
    )
    # Predict on validation set
    y_valid_pred_proba = lgbm_model.predict_proba(X_valid)[:, 1]
    
    # Evaluate model on validation set
    valid_roc_auc = roc_auc_score(y_valid, y_valid_pred_proba)
    print(f'Validation ROC AUC: {valid_roc_auc}')
    valid_roc_aucs.append(valid_roc_auc)

print(f'BASE LINE -Overall Validation ROC AUC is: {np.mean(valid_roc_aucs)} ± {np.std(valid_roc_aucs)}')

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 23451, number of negative: 740032
[LightGBM] [Info] Total Bins 9237
[LightGBM] [Info] Number of data points in the train set: 763483, number of used features: 133
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.030716 -> initscore=-3.451780
[LightGBM] [Info] Start training from score -3.451780
Validation ROC AUC: 0.6976373009689332
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 24543, number of negative: 738633
[LightGBM] [Info] Total Bins 9026
[LightGBM] [Info] Number of data points in the train set: 763176, number of used features: 133
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.032159 -> initscore=-3.404374
[LightGBM] [Info] Start t

BASE LINE -Overall Validation ROC AUC is: 0.699815746487821 ± 0.002

In [20]:
cv = GroupKFold(n_splits=3)

# Initialize list to store results
valid_roc_aucs = []

# Initialize the LGBMClassifier
lgbm_model = LGBMClassifier(
    force_col_wise=True, 
    random_state=42,
    extra_trees=True,
    n_estimators=2000,
    num_leaves=54,
    boosting_type='gbdt',
    objective='binary',
    learning_rate=0.03,
    l1_regularization=0.1,
    l2_regularization=10,
    max_depth=-1,
    min_data_in_leaf=20,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=5,
    verbose=-1
)

# Perform GroupKFold cross-validation
for idx_train, idx_valid in cv.split(X_scaled, y, groups=weeks):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
    # Train the model
    lgbm_model.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_metric='auc',
        callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=100)]
    )
    # Predict on validation set
    y_valid_pred_proba = lgbm_model.predict_proba(X_valid)[:, 1]
    
    # Evaluate model on validation set
    valid_roc_auc = roc_auc_score(y_valid, y_valid_pred_proba)
    valid_roc_aucs.append(valid_roc_auc)

    print(f'Fold Validation complex model ROC AUC: {valid_roc_auc}')

# Output overall validation results
print(f'Overall Validation Complex Model ROC AUC: {np.mean(valid_roc_aucs)} ± {np.std(valid_roc_aucs)}')

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[675]	training's auc: 0.748878	training's binary_logloss: 0.124052	valid_1's auc: 0.694791	valid_1's binary_logloss: 0.134517
Fold Validation complex model ROC AUC: 0.6947911977409509
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[883]	training's auc: 0.751822	training's binary_logloss: 0.124909	valid_1's auc: 0.705666	valid_1's binary_logloss: 0.130817
Fold Validation complex model ROC AUC: 0.7056661063375721
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[771]	training's auc: 0.746455	training's binary_logloss: 0.127032	valid_1's auc: 0.713343	valid_1's binary_logloss: 0.127197
Fold Validation complex model ROC AUC: 0.7133431623316913
Overall Validation Complex Model ROC AUC: 0.7046001554700715 ± 0.007611221364229935


In [23]:
lgbm_model.fit(
    X_scaled, y,
    eval_metric='auc',
)

LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, extra_trees=True,
               feature_fraction=0.8, force_col_wise=True, l1_regularization=0.1,
               l2_regularization=10, learning_rate=0.03, min_data_in_leaf=20,
               n_estimators=2000, num_leaves=54, objective='binary',
               random_state=42, verbose=-1)

the complexity of the classifier does not provide the additional value for our predictions, my oppinion is to try forward the ensembels techniques for improvement teh efficiency of the oredictins, or to return to base model which shows robust results on all cross_folds ±0.0002
BASE LINE -Overall Validation ROC AUC is: 0.6998

In [26]:
X_test = test_data.drop(columns=['WEEK_NUM', 'target']).set_index('case_id')

# Predict on test set (assuming you have a separate test set)
X_test_scaled = scaler.transform(X_test)
y_test_pred_proba = lgbm_model.predict_proba(X_test_scaled)[:, 1]

sample = pd.read_csv('/Users/nataliamarko/Documents/Kaggle/home_credit_risk_score/sample_submission.csv')
sample = sample.set_index('case_id')
predictions = pd.DataFrame(y_test_pred_proba, index=X_test.index)

# Assign the predictions to the 'score' column in the sample submission
sample['score'] = predictions
print(sample)

sample.to_csv("submission.csv")
gc.collect()

            score
case_id          
57543    0.130243
57549    0.113207
57551    0.140332
57552    0.140332
57569    0.137036
57630    0.133266
57631    0.142977
57632    0.124476
57633    0.129617
57634    0.126474


872